<a href="https://colab.research.google.com/github/flyaflya/persuasive/blob/main/demoNotebooks/carsFixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install matplotlib numpyro daft --upgrade  ##restart runtime

## Get the Data

In [1]:
import pandas as pd
url = "https://raw.githubusercontent.com/flyaflya/persuasive/main/carsFixed.csv"
carsDF = pd.read_csv(url)
carsDF

,observation,shopID,boss,carsFixed
0,1,1,0,8
1,2,2,0,22
2,3,3,0,32
3,4,4,1,64
4,5,5,0,53
...,...,...,...,...
245,246,1,1,18
246,247,2,0,22
247,248,3,0,31
248,249,4,0,26


In [4]:
(
    carsDF
    .groupby(carsDF.shopID)
    .agg(numObservations=('observation', 'count'), 
        numBossVisits =('boss', 'sum'))
)

,numObservations,numBossVisits
shopID,,
1,50,10
2,50,5
3,50,15
4,50,5
5,50,15


## Using carsDF, create a numpyro model for cars fixed by shop
Assume carsFixed is poisson, ignore the effect of the boss.  What estimates do you get for lambda?